## Hyperparameter search

Source: https://github.com/itdxer/neupy/blob/master/notebooks/Hyperparameter%20optimization%20for%20Neural%20Networks.ipynb
and http://neupy.com/2016/12/17/hyperparameter_optimization_for_neural_networks.html#id14

```python
Hyperparams to train:
1. lamda
2. number of hidden layers
3. number of hidden units
3. starter learning rate
5. use_sigmoid_h1
6. use_sigmoid_h2```

df_tfs_random = gv.df_tfs
df_dhss_random = gv.df_dhss

In [ ]:
sns.kdeplot(df_tfs_random.index.get_level_values("pcc"), color="orange", label="random tfs")
sns.kdeplot(gv.df_tfs.index.get_level_values("pcc"), color="red", label="real tfs")

In [ ]:
gv.df_tfs

In [ ]:
import time
import os
import sys
sys.path = sys.path[1:]
sys.path.insert(0, os.path.join(os.getcwd(), "helper_scripts"))  # pretending we are running main.py in jupyter

from global_variables_final_for_git import Global_Vars
from prep_for_model_for_git import Model_preparation

import hyperopt
from hyperopt import fmin
import HPO_helper
reload(HPO_helper)
from HPO_helper import uniform_int, loguniform_int, tpe_method, get_parameter_space_forHPO
import tensorflow_model_for_git
reload(tensorflow_model_for_git)
from tensorflow_model_for_git import Tensorflow_model

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
class Args(object):
    def __init__(self):
        self.gene = "MYOD1"
        self.distance = 200
        self.use_tad_info = True
        self.pcc_lowerlimit_to_filter_dhss = 0.1
        self.take_log2_tpm = True
        self.filter_tfs_by = "zscore" # or "pcc"
        self.lowerlimit_to_filter_tfs = 4.75
        self.take_this_many_top_fts = 15  # all dhss/tfs will already be filtered by pcc(or zscore)
        self.init_wts_type = "corr"
        self.outputDir = "/Users/Dinesh/Dropbox/Github/predicting_gex_with_nn_git/Output/"+self.gene.upper()
        self.use_random_DHSs = True
        self.use_random_TFs = True
        self.max_iter = 300

start_time = time.time()
args = Args()
gv = Global_Vars(args, args.outputDir)  # note this takes in new_output_dir as well in .py scripts
mp = Model_preparation(gv)

2018-01-22 15:31:53,431: global_variables_final_for_git: INFO    : Setting up the DNase-seq dataframe and gene expression vector..


> /Users/Dinesh/Dropbox/Github/predicting_gex_with_nn_git/Functions/model_scripts/helper_scripts/global_variables_final_for_git.py(288)get_random_df_tfs_filtdBy_pcc_and_size()
-> rand_ints = sorted(random.sample(range(0, df_random.shape[0]), max_tfs_num))
(Pdb) df_random.shape
(1230, 129)
(Pdb) df_random.head(2)
                                                  E017_LNG.IMR90_IMR90  E002_ESC.WA7_ESC  E008_ESC.H9_ESC  E001_ESC.I3_ESC  E015_ESC.HUES6_ESC  E014_ESC.HUES48_ESC  E016_ESC.HUES64_ESC  E003_ESC.H1_ESC  E024_ESC.4STAR_ESC  E020_IPSC.20B_iPSC    ...     E122_VAS.HUVEC_ENCODE2012  E123_BLD.K562.CNCR_ENCODE2012  E124_BLD.CD14.MONO_ENCODE2012  E125_BRN.NHA_ENCODE2012  E126_SKIN.NHDFAD_ENCODE2012  E127_SKIN.NHEK_ENCODE2012  E128_LNG.NHLF_ENCODE2012  E129_BONE.OSTEO_ENCODE2012       pcc   abs_pcc
geneName loc                  TAD_loc                                                                                                                                                         

In [ ]:

'''Run HPO on differen train/test splits'''
for test_idx in range(0, 1):
    # test_idx = 6  # 5 corresponds to ES_deriv group
    if (test_idx == 4):  # corresponds to val_group of "ENCODE2012"
        continue
    tm = Tensorflow_model(gv, mp, test_eid_group_index=test_idx)
    trials = hyperopt.Trials()

    best_params = hyperopt.fmin(tm.train_tensorflow_nn, trials=trials, space=get_parameter_space_forHPO(tm.trainX),
                                algo=tpe_method, max_evals=10) 

    med_pc_test_error, med_pc_val_error = tm.plot_scatter_performance(trials, gv, index=None)
    med_val_pcc = trials.results[np.argmin(trials.losses())]["val_pcc"].flatten()[-1]
    logger.info("trainX.shape:{}, testX.shape:{}".format(tm.trainX.shape, tm.testX.shape))
    logger.info("Test Group {}: {},\
                Median Test pc Error: {},\
                Median Val (pc error, pcc): ({},{})\
                Best Params: {}".format(
        tm.test_eid_group_index, tm.test_eid_group,
        round(med_pc_test_error, 3),
        round(med_pc_val_error, 3), round(med_val_pcc, 3),
        best_params))
    del tm, trials, best_params

print("Total time taken: {}".format(time.time() - start_time))

# DeepLIFT implementation

trial_idx = 1
plt.plot(trials.results[trial_idx]['train_loss'], color="salmon")
plt.plot(trials.results[trial_idx]['val_loss'], color="mediumvioletred")

In [ ]:
# - EOF - 